In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from tqdm import tqdm
import gc
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score
tqdm.pandas()
pd.set_option("display.max_columns", None)

train_df = pd.read_parquet("task3_recsys/train_df_after_transformer.parquet")
test_df = pd.read_parquet("task3_recsys/test_df_after_transformer.parquet")
numeric_df = pd.read_parquet("task3_recsys/numeric_df_for_booster.parquet")

Сделаем признаки для бустинга, помимо эмбеддингов трансформеров.

In [2]:
numeric_df["top5_most_common_mcc"] = numeric_df.mcc_cat_list.progress_apply(lambda x: Counter(x).most_common(5))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 963811/963811 [00:16<00:00, 57852.75it/s]


In [3]:
numeric_df["days_b_list_std"] = numeric_df.days_b_list.map(np.std)
numeric_df["days_b_list_mean"] = numeric_df.days_b_list.map(np.mean)
numeric_df["days_b_list_min"] = numeric_df.days_b_list.map(min)
numeric_df["days_b_list_max"] = numeric_df.days_b_list.map(max)
numeric_df["total_trx"] = numeric_df.tr_order_list.map(len)
numeric_df = numeric_df.drop(columns=["days_b_list", "tr_order_list"])

In [4]:
for i in tqdm(range(1, 6)):
    i_real = i - 1
    numeric_df[f"mcc_top{i}_class"] = numeric_df.top5_most_common_mcc.apply(lambda x: x[i_real][0] if len(x) > i_real else -1)
    numeric_df[f"mcc_top{i}_counts"] = numeric_df.top5_most_common_mcc.apply(lambda x: x[i_real][1] if len(x) > i_real else -1)

    numeric_df[f"amnt_temp_{i}"] = numeric_df.apply(lambda row: row.amnt_list[(row.mcc_cat_list == row[f"mcc_top{i}_class"])] if row[f"mcc_top{i}_class"] else [], axis=1)

    numeric_df[f"mcc_top{i}_amount_mean"] = numeric_df.apply(lambda row: row[f"amnt_temp_{i}"].mean() if len(row[f"amnt_temp_{i}"]) else -1, axis=1)
    numeric_df[f"mcc_top{i}_amount_std"] = numeric_df.apply(lambda row: row[f"amnt_temp_{i}"].std() if len(row[f"amnt_temp_{i}"]) else -1, axis=1)
    numeric_df[f"mcc_top{i}_amount_max"] = numeric_df.apply(lambda row: row[f"amnt_temp_{i}"].max() if len(row[f"amnt_temp_{i}"]) else -1, axis=1)
    numeric_df[f"mcc_top{i}_amount_min"] = numeric_df.apply(lambda row: row[f"amnt_temp_{i}"].min() if len(row[f"amnt_temp_{i}"]) else -1, axis=1)

numeric_df = numeric_df.drop(columns=[f"amnt_temp_{i}" for i in range(1, 6)])

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [02:56<00:00, 35.37s/it]


In [5]:
numeric_df["last_mcc"] = numeric_df.mcc_cat_list.apply(lambda x: x[-1])
numeric_df["amnt_list_temp"] = numeric_df.amnt_list.apply(lambda x: x[np.nonzero(x)])

numeric_df["amnt_list_std"] = numeric_df.amnt_list_temp.apply(lambda x: np.std(x) if len(x) else -1)
numeric_df["amnt_list_mean"] = numeric_df.amnt_list_temp.apply(lambda x: np.mean(x) if len(x) else -1)
numeric_df["amnt_list_min"] = numeric_df.amnt_list_temp.apply(lambda x: np.min(x) if len(x) else -1)
numeric_df["amnt_list_max"] = numeric_df.amnt_list_temp.apply(lambda x: np.max(x) if len(x) else -1)

In [6]:
numeric_df = numeric_df.drop(columns=["amnt_list_temp", "mcc_cat_list", "amnt_list", "top5_most_common_mcc"])

In [7]:
train_df_full = numeric_df.merge(train_df, how="inner", on="app_id")
test_df_full = numeric_df.merge(test_df, how="inner", on="app_id")
del numeric_df, train_df, test_df
gc.collect();

In [8]:
train_df_full = train_df_full.drop(columns=["mcc_cat_list", "amnt_list_discrete", "next_mcc_y"]).rename(columns={"next_mcc_x": "Y"})
test_df_full = test_df_full.drop(columns=["mcc_cat_list", "amnt_list_discrete", "next_mcc_y"]).rename(columns={"next_mcc_x": "Y"})

In [9]:
train_df_full.head(5)

,app_id,Y,days_b_list_std,days_b_list_mean,days_b_list_min,days_b_list_max,total_trx,mcc_top1_class,mcc_top1_counts,mcc_top1_amount_mean,mcc_top1_amount_std,mcc_top1_amount_max,mcc_top1_amount_min,mcc_top2_class,mcc_top2_counts,mcc_top2_amount_mean,mcc_top2_amount_std,mcc_top2_amount_max,mcc_top2_amount_min,mcc_top3_class,mcc_top3_counts,mcc_top3_amount_mean,mcc_top3_amount_std,mcc_top3_amount_max,mcc_top3_amount_min,mcc_top4_class,mcc_top4_counts,mcc_top4_amount_mean,mcc_top4_amount_std,mcc_top4_amount_max,mcc_top4_amount_min,mcc_top5_class,mcc_top5_counts,mcc_top5_amount_mean,mcc_top5_amount_std,mcc_top5_amount_max,mcc_top5_amount_min,last_mcc,amnt_list_std,amnt_list_mean,amnt_list_min,amnt_list_max,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255
0,32,2,104.949155,155.441899,1,359,611,4,253,0.306038,0.036479,0.424749,0.192694,1,137,0.329665,0.039944,0.427843,0.229750,3,75,0.310373,0.033390,0.367847,0.220630,2,72,0.424111,0.083555,0.616428,0.235098,9,16,0.399837,0.048436,0.486348,0.268832,3,0.062288,0.333065,0.192694,0.616428,-0.045389,-0.205600,-0.128679,-0.120213,0.156994,0.053706,0.087332,-0.109532,-0.003793,0.103276,-0.128244,-0.069607,-0.051354,-0.180390,0.111737,-0.094453,-0.030913,0.001875,-0.128192,-0.060380,-0.074507,-0.055055,0.023410,0.016163,-0.070287,0.094856,0.176770,-0.244736,0.015249,0.108112,0.243645,0.182034,0.215546,0.100968,0.110819,0.142000,0.131631,0.109563,-0.122887,-0.118338,-0.331012,0.003391,-0.201983,-0.018292,-0.067320,0.065449,-0.014096,0.051266,0.092255,0.017338,0.161364,0.240438,-0.060510,-0.207890,-0.093277,0.072365,0.162034,0.287715,0.050252,-0.307555,0.086968,-0.161917,0.074911,0.052735,-0.061006,0.031616,-0.045064,0.056697,0.223555,0.289237,0.087247,-0.057105,-0.179519,-0.262923,-0.136821,-0.162574,0.025824,0.672627,-0.185175,0.079805,0.240562,-0.126162,0.162406,-0.455612,-0.339530,0.145511,-0.125572,-0.238858,0.017952,-0.046779,0.071908,0.055060,-0.029269,-0.625583,-0.624451,0.068029,-0.179680,0.116192,0.165789,-0.207811,-0.191838,-0.155610,-0.104278,-0.065338,0.077955,0.354811,0.162101,-0.158598,-0.251037,-0.525141,-0.251772,0.404343,0.371394,0.029830,-0.044137,0.387374,-0.243611,-0.068973,-0.402699,-0.008902,-0.129672,-0.352656,-0.364832,-0.346593,0.030627,-0.151452,0.259775,0.590017,-0.434933,0.024074,-0.090985,-0.164926,-0.144073,-0.045595,0.220820,-0.002754,0.184135,0.160620,0.009377,-0.074345,0.415731,0.192814,-0.072256,0.571472,0.017911,0.313754,-0.136437,-0.052933,0.338181,0.023110,-0.031553,-0.209530,0.151198,0.246306,-0.224522,0.477900,-0.193621,0.175087,0.041343,0.137944,-0.156913,0.588284,-0.292154,0.069743,-0.145511,-0.587554,-0.054382,0.247743,0.535168,0.325918,-0.040965,-0.095664,0.353560,0.038087,0.238358,0.098332,-0.117848,0.045565,-0.012115,-0.031422,0.421710,0.218938,-0.510059,0.247175,-0.352057,-0.138940,0.339217,-0.055332,-0.253148,0.076731,0.426402,-0.250764,0.219061,-0.159094,0.430633,-0.081049,-0.300646,0.401079,0.032357,0.026155,-0.013248,0.023442,-0.166490,0.155307,-0.117719,0.181984,-0.491777,-0.061884,-0.261308,-0.059753,-0.151791,0.590393,-0.010769,-0.099736,-0.347035,0.030337,0.166582,0.311192,0.152476,0.381892,

In [10]:
train_df_full.shape[0] + test_df_full.shape[0]

963800

Имеем немало категориальных переменных, откуда проще будет использовать Catboost:

In [11]:
cat_feats = [f"mcc_top{i}_class" for i in range(1, 6)] + ["last_mcc"]

Xtrain, Ytrain = train_df_full.drop(columns=["app_id", "Y"]), train_df_full[["Y"]]
Xtest, Ytest = test_df_full.drop(columns=["app_id", "Y"]), test_df_full[["Y"]]

In [14]:
model = CatBoostClassifier(
    iterations=256, 
    max_depth=6,
    task_type='GPU',
    devices='0',
    auto_class_weights="SqrtBalanced"
    )
model.fit(Xtrain, Ytrain, cat_features=cat_feats)

Learning rate set to 0.5
0:	learn: 2.9063504	total: 247ms	remaining: 1m 2s
1:	learn: 11.5563640	total: 491ms	remaining: 1m 2s
2:	learn: 20.1278156	total: 695ms	remaining: 58.6s
3:	learn: 19.7282576	total: 925ms	remaining: 58.3s
4:	learn: 19.6561708	total: 1.18s	remaining: 59.1s
5:	learn: 16.6695589	total: 1.4s	remaining: 58.5s
6:	learn: 17.5004033	total: 1.63s	remaining: 57.9s
7:	learn: 17.4709892	total: 1.88s	remaining: 58.2s
8:	learn: 16.9390640	total: 2.09s	remaining: 57.4s
9:	learn: 16.5008352	total: 2.32s	remaining: 57.1s
10:	learn: 16.4277639	total: 2.56s	remaining: 57.1s
11:	learn: 16.3789743	total: 2.76s	remaining: 56.1s
12:	learn: 12.2905950	total: 3s	remaining: 56s
13:	learn: 18.8354004	total: 3.22s	remaining: 55.6s
14:	learn: 16.2370419	total: 3.45s	remaining: 55.5s
15:	learn: 20.7079038	total: 3.68s	remaining: 55.2s
16:	learn: 20.2428836	total: 3.91s	remaining: 55s
17:	learn: 20.2232231	total: 4.16s	remaining: 55s
18:	learn: 20.1709287	total: 4.42s	remaining: 55.1s
19:	lear

In [17]:
from sklearn.metrics import classification_report, accuracy_score, f1_score

for data, label, name in [(Xtrain, Ytrain, 'Train'), (Xtest, Ytest, 'Test')]:
    preds = model.predict(data)
    print(name)
    # print(classification_report(label, preds))
    print('Accuracy:', accuracy_score(label, preds))
    print('F1 Weighted:', f1_score(label, preds, average='weighted'))
    print('F1 Macro:', f1_score(label, preds, average='macro'))
    print("===")

Train
Accuracy: 0.43771269973023447
F1 Weighted: 0.44764893974176323
F1 Macro: 0.20499396047192867
===
Test
Accuracy: 0.43191533513177005
F1 Weighted: 0.44191730374287164
F1 Macro: 0.18090346334763271
===


Бейзлайн (самая популярная категория):

In [30]:
Ybaseline = Ytest.copy()
Ybaseline["Y"] = Ytest.value_counts("Y").index[0]

print("Test baseline - just most popular mcc")
# print(classification_report(Ytest, Ybaseline))
print('Accuracy:', accuracy_score(Ytest, Ybaseline))
print('F1 Weighted:', f1_score(Ytest, Ybaseline, average='weighted'))
print('F1 Macro:', f1_score(Ytest, Ybaseline, average='macro'))

Test baseline - just most popular mcc
Accuracy: 0.36336376841668394
F1 Weighted: 0.19368745342456584
F1 Macro: 0.019742225311437114


Откуда, можем сделать вывод, что с кол-вом классов, равным 28, при этом с сильно выраженным неравномерным распределением саппорта, наша модель справляется значительно лучше бейзлайна.

Посмотрим на Feature importance:

In [37]:
pd.DataFrame([{k:v for k, v in zip(model.feature_names_, model.feature_importances_)}]).T.sort_values(0, ascending=False).rename(columns={0: "Gain"}).iloc[:20]

,Gain
days_b_list_min,28.318725
219,20.755518
mcc_top1_amount_mean,13.180662
225,11.208975
143,11.068280
44,6.580822
last_mcc,4.346994
mcc_top5_amount_min,2.841254
mcc_top5_counts,1.548331
mcc_top5_amount_std,0.036743
